In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [35]:
# list_mz_index = [] ;
# # 抹账数据处理
# df_data_抹账=df_data_交易[df_data_交易['账户代号'].isin(df_data_交易[df_data_交易['交易金额']<0]['账户代号'].unique())].sort_values(by=['账户代号','交易日期','交易时间'],ascending=True).reset_index() ;
# #待剔除数据
# tc_idx=set()
# for idx in df_data_抹账[df_data_抹账['交易金额']<0].index:
#     for d in range(1,5):
#         #print(idx)
#         if -df_data_抹账.loc[idx,'交易金额']==df_data_抹账.loc[idx-d,'交易金额'] and df_data_抹账.loc[idx,'账户代号']==df_data_抹账.loc[idx-d,'账户代号']\
#             and df_data_抹账.loc[idx,'对方账号']==df_data_抹账.loc[idx-d,'对方账号']:
#                 list_mz_index.append(idx-d) ;
#                 tc_idx.add(idx)
#                 tc_idx.add(idx-d)
#                 break
# tc_idx=list(tc_idx)
# df_data_交易_去除抹账后=df_data_抹账[~df_data_抹账.index.isin(tc_idx)]
# df_data_交易_去除抹账后=pd.concat([df_data_交易_去除抹账后,df_data_交易[df_data_交易['交易金额']>=0]])

In [7]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [8]:
df_data_交易_去除抹账后.shape

(812264, 12)

In [9]:
df_data_交易.shape

(816270, 12)

In [10]:
816270 - 812264

4006

In [22]:
clicode = "2DB552D3B7D0FAAD"
df1 = df_data_交易_去除抹账后[df_data_交易_去除抹账后["账户代号"] == clicode].sort_values(by=["交易日期","交易时间"])

In [23]:
df1

,交易流水序号,账户代号,对方账号,借贷标志,交易金额,交易余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度
11510,906EA1B378282A3431008A2570E294F0,2DB552D3B7D0FAAD,68A76D9C06006353,0,15172.90,21572.63,B720B8DA,2020-03-02,08:00:01,091D584F,A97DA629,9
27761,1F8505D17A13F7A5D156442585E99DDD,2DB552D3B7D0FAAD,BA8C172953D66632,1,2045.00,23617.63,D41D8CD9,2020-03-03,22:19:18,757B505C,D516B136,33
27772,E06C8EBF41304BB5D1970628510EA57D,2DB552D3B7D0FAAD,C61831BC48554375,0,1860.28,21757.35,8B09A86D,2020-03-03,22:21:54,091D584F,A97DA629,9
27789,C0C7E5058B554BBC66F9B1243692C07A,2DB552D3B7D0FAAD,EDF7480758B236D5,0,2206.63,19550.72,A71C76B8,2020-03-03,22:23:16,091D584F,A97DA629,9
36455,CA4761DE7D3081B8488721AE6468E832,2DB552D3B7D0FAAD,B8C1FDFA9113CDCE,0,7200.00,12350.72,B7D3CD1F,2020-03-05,06:40:48,091D584F,A97DA629,9
45772,4BED0B7CE309F4F71196C9B6B66C7709,2DB552D3B7D0FAAD,14BEFED1370B730A,0,4966.37,7384.35,834E1F06,2020-03-06,09:26:20,E96ED478,4E0CB6FB,45
95412,CBDB29C1FE9FF45D728C12F19DCAD9B1,2DB552D3B7D0FAAD,81FB0A3FC7B23FC4,0,1.00,7383.35,75AC9CB0,2020-03-11,03:10:11,892C91E0,38DB3AED,24
134139,EA9E94986A00C2DE0CD73A730CB5D144,2DB552D3B7D0FAAD,F589634A79D1CD4B,0,6849.92,533.43,45D69992,2020-03-15,05:03:05,621461AF,068815E0,9
182464,C0D1DBAEC3D2E17F67A3CDED32F63CE8,2DB552D3B7D0FAAD,B8C1FDFA9113CDCE,1,800.00,1333.43,B7D3CD1F,2020-03-19,21:36:41,621461AF,A3C65C29,9
182476,78C907AA963AFD48E4F25168C0FCBE9E,2DB552D3B7D0FAAD,F6E026C6DBD21350,0,1240.00,93.43,28300C82,2020-03-19,21:38:34,091D584F,A97DA629,9


In [16]:
df1[df1["借贷标志"] == 0].groupby("对方账号")["交易金额" , "交易余额"].agg(["mean" , "max" , "min", "count"]).sort_values(by=[("交易金额" , "count")] , ascending=False)
# df1


C:\Users\14714\AppData\Local\Temp\ipykernel_18356\819532114.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1[df1["借贷标志"] == 0].groupby("对方账号")["交易金额" , "交易余额"].agg(["mean" , "max" , "min", "count"]).sort_values(by=[("交易金额" , "count")] , ascending=False)


交易金额                               交易余额                        
                      mean       max       min count     mean      max      min count
对方账号                                                                                 
381713A874F52E77   3285.71   5000.00   2000.00     7 18607.48 41677.01  1180.33     7
5BB4A1253D1858C6      1.00      1.00      1.00     3    23.58    36.37     7.01     3
0955F9CB682D8625   3690.00   5000.00   2380.00     2 25487.01 26677.01 24297.01     2
14BEFED1370B730A    326.98    501.99    151.96     2  8101.70 16175.02    28.37     2
31668907BACD081A   2125.00   2750.00   1500.00     2 15676.01 16677.01 14675.02     2
8A9BFF87D530C602   6390.00   6390.00   6390.00     2    91.87   146.37    37.37     2
97DCA96389776547   1500.00   2000.00   1000.00     2 22428.67 44677.01   180.33     2
A749AD11159A920C   2435.00   2620.00   2250.00     2 20552.01 21677.01 19427.01     2
4841FD47EE9042F6 360000.00 360000.00 360000.00     1 90007.01 90007.01 90007.01     1
57E8143D8F5DA4A1   3458.00   3458.00   3458.00     1   186.37   186.37   186.37     1
69A0FB2E3F6FF385  43330.00  43330.00  43330.00     1 46677.01 46677.01 46677.01     1
99088DDCBCCF1129   3500.00   3500.00   3500.00     1 11175.02 11175.02 11175.02     1